## Step 1: requirements & library install (customizable)

In [ ]:
# requirements.txt
!pip install -q tensorflow openvino openvino-dev[onnx] --no-deps opencv-python Pillow thinc numpy datasets

: 

## Step 2: Drive connect

In [4]:
# 2. Impor semua pustaka yang diperlukan
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import openvino as ov
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from google.colab import drive

# 3. Hubungkan ke Google Drive
drive.mount('/content/drive')
print("\n✅ Setup selesai. Semua pustaka berhasil diimpor dan Google Drive terhubung.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Setup selesai. Semua pustaka berhasil diimpor dan Google Drive terhubung.


## Step 3: Configruation

In [5]:
# ==============================================================================
# SEL 1: KONFIGURASI & MISC
# ==============================================================================

import tensorflow as tf
import os

# Aktifkan Mixed Precision untuk akselerasi (L4 juga mendukung)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Konfigurasi training yang dioptimalkan untuk L4
CFG = {
    # Path Google Drive untuk model & label
    "gdrive_model_dir": "/content/drive/MyDrive/AgrifyAI/models/",

    # Simpan & load model
    "model_path": "/content/drive/MyDrive/AgrifyAI/models/plant_disease_model_L4.keras",  # full model
    "weights_path": "/content/drive/MyDrive/AgrifyAI/models/plant_disease_model_L4.weights.h5",   # weights only
    "openvino_path_xml": "/content/drive/MyDrive/AgrifyAI/models/openvino_model.xml",
    "openvino_path_bin": "/content/drive/MyDrive/AgrifyAI/models/openvino_model.bin",

    # Ekspor alternatif
    "onnx_path": "/content/drive/MyDrive/AgrifyAI/models/plant_disease_model_L4.onnx",

    # Label path
    "all_label_path": "/content/drive/MyDrive/AgrifyAI/models/labels.json",
    "translated_label_path": "/content/drive/MyDrive/AgrifyAI/models/labels_id.json",
    "plantvillage_labels": "/content/drive/MyDrive/AgrifyAI/models/plantvillage_labels.json",
    "paddy_labels": "/content/drive/MyDrive/AgrifyAI/models/paddy_labels.json",

    # History training
    "history_path": "/content/drive/MyDrive/AgrifyAI/models/history_L4.json",

    # Parameter training
    "img_size": (224, 224),
    "batch_size": 128,          # Disesuaikan dengan VRAM L4
    "epochs": 20,
    "lr": 1e-4,
    "patience": 5,
    "min_lr": 1e-6,
    "fine_tune_at": 100,
    "fine_tune_epochs": 10,
    "fine_tune_lr": 1e-5,
    "validation_split": 0.2,
}


# Membuat direktori jika belum ada
os.makedirs(CFG["gdrive_model_dir"], exist_ok=True)

print(f"✅ Konfigurasi L4 siap. Batch size: {CFG['batch_size']}, Total Epochs: {CFG['epochs'] + CFG['fine_tune_epochs']}")
print("⚡️ Mixed Precision 'mixed_float16' telah diaktifkan.")


✅ Konfigurasi L4 siap. Batch size: 128, Total Epochs: 30
⚡️ Mixed Precision 'mixed_float16' telah diaktifkan.


## Step 4: The Datasets

In [6]:
# ==============================================================================
# SEL 2 (REFAC): LOAD 2 DATASET SEKALIGUS, GABUNG LABELS
# ==============================================================================

from datasets import load_dataset
import tensorflow as tf
import json, os, numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

# --- Augmentasi ringan ---
ADVANCED_AUGMENTATION = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="light_augmentation")

# --- Cutout ---
def cutout(image, label, p=0.3, ratio=0.2):
    if tf.random.uniform([]) > p:
        return image, label

    h, w = tf.shape(image)[0], tf.shape(image)[1]
    cut_h = tf.cast(tf.cast(h, tf.float32) * ratio, tf.int32)
    cut_w = tf.cast(tf.cast(w, tf.float32) * ratio, tf.int32)

    cy = tf.random.uniform([], 0, h, dtype=tf.int32)
    cx = tf.random.uniform([], 0, w, dtype=tf.int32)

    y1 = tf.clip_by_value(cy - cut_h // 2, 0, h)
    y2 = tf.clip_by_value(cy + cut_h // 2, 0, h)
    x1 = tf.clip_by_value(cx - cut_w // 2, 0, w)
    x2 = tf.clip_by_value(cx + cut_w // 2, 0, w)

    mask = tf.ones((h, w), dtype=image.dtype)
    mask = tf.tensor_scatter_nd_update(
        mask,
        indices=tf.reshape(tf.stack(tf.meshgrid(tf.range(y1, y2), tf.range(x1, x2)), -1), (-1, 2)),
        updates=tf.zeros(((y2-y1)*(x2-x1),), dtype=image.dtype)
    )
    return image * tf.expand_dims(mask, axis=-1), label

# --- Preprocess per contoh ---
def preprocess_example(example, cfg, image_col, label_col):
    img = example[image_col]
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = tf.convert_to_tensor(np.array(img), dtype=tf.float32)
    img = tf.image.resize(img, cfg["img_size"])
    img = preprocess_input(img)
    label = tf.convert_to_tensor(example[label_col], dtype=tf.int64)
    return img, label

def preprocess_with_aug(image, label, augment=True):
    if augment:
        image = ADVANCED_AUGMENTATION(image)
        image, label = cutout(image, label)
    return image, label

def hf_to_tf(dataset_split, cfg, image_col, label_col):
    return tf.data.Dataset.from_generator(
        lambda: (preprocess_example(ex, cfg, image_col, label_col) for ex in dataset_split),
        output_signature=(
            tf.TensorSpec(shape=cfg["img_size"] + (3,), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )

# --- Pipeline utama untuk load banyak dataset ---
def create_combined_dataset(cfg, datasets_config, augment=True):
    """
    datasets_config = dict of { "pv": config_dict, "paddy": config_dict }
    """
    all_labels_map = {}
    train_tf_datasets = []
    val_tf_datasets = []

    for ds_key, ds_cfg in datasets_config.items():
        # Load HF dataset
        dataset_full = load_dataset(ds_cfg['name'], split="train")
        class_names = dataset_full.features[ds_cfg['label_col']].names
        num_classes = len(class_names)
        # Split train/val
        dataset_split = dataset_full.train_test_split(test_size=cfg["validation_split"], seed=42)

        # Convert ke tf.data.Dataset
        train_ds = hf_to_tf(dataset_split["train"], cfg, ds_cfg['image_col'], ds_cfg['label_col'])
        val_ds   = hf_to_tf(dataset_split["test"], cfg, ds_cfg['image_col'], ds_cfg['label_col'])

        AUTOTUNE = tf.data.AUTOTUNE
        train_ds = train_ds.map(lambda x, y: preprocess_with_aug(x, y, augment), num_parallel_calls=AUTOTUNE)\
                           .shuffle(1024).batch(cfg["batch_size"], drop_remainder=True).prefetch(AUTOTUNE)
        val_ds   = val_ds.map(lambda x, y: preprocess_with_aug(x, y, augment), num_parallel_calls=AUTOTUNE)\
                         .batch(cfg["batch_size"]).prefetch(AUTOTUNE)

        train_tf_datasets.append(train_ds)
        val_tf_datasets.append(val_ds)

        # Simpan label sementara
        all_labels_map.update({f"{ds_key}_{i}": n for i, n in enumerate(class_names)})

    # Gabung dataset tf.data (opsional)
    combined_train = train_tf_datasets[0]
    combined_val   = val_tf_datasets[0]
    for ds in train_tf_datasets[1:]:
        combined_train = combined_train.concatenate(ds)
    for ds in val_tf_datasets[1:]:
        combined_val = combined_val.concatenate(ds)

    # Simpan label gabungan
    label_path = os.path.join(cfg["gdrive_model_dir"], "labels.json")
    with open(label_path, "w") as f:
        json.dump(all_labels_map, f, indent=2)

    print(f"✅ Combined dataset siap. Label disimpan di: {label_path}")
    return combined_train, combined_val, all_labels_map

# --- Contoh penggunaan ---
datasets_config = {
    "pv": {
        'name': 'BrandonFors/Plant-Diseases-PlantVillage-Dataset',
        'image_col': 'image',
        'label_col': 'label'
    },
    "paddy": {
        'name': 'anthony2261/paddy-disease-classification',
        'image_col': 'image',
        'label_col': 'label'
    }
}

train_dataset, val_dataset, labels_map = create_combined_dataset(CFG, datasets_config, augment=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/170M [00:00<?, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002-a929229f3aaf71(…):   0%|          | 0.00/402M [00:00<?, ?B/s]

data/train-00001-of-00002-4c2a20b9469e90(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10407 [00:00<?, ? examples/s]

✅ Combined dataset siap. Label disimpan di: /content/drive/MyDrive/AgrifyAI/models/labels.json


## Additional Step: Merge & Translate Label

In [7]:
import json
import os

def merge_labels(cfg):
    """
    Merge label PlantVillage & Paddy Disease menjadi satu all_labels.json.
    Hasil merge disimpan di cfg['all_label_path'].
    """
    all_labels = {}

    # --- Load masing-masing label ---
    for key_name, path in [('PlantVillage', cfg.get('plantvillage_labels')),
                           ('Paddy', cfg.get('paddy_labels'))]:
        if path and os.path.exists(path):
            with open(path, 'r', encoding='utf-8') as f:
                labels = json.load(f)
            # Tambahkan prefix supaya tidak bentrok key (opsional)
            for k, v in labels.items():
                new_key = f"{key_name}_{k}"
                all_labels[new_key] = v
            print(f"✅ Label {key_name} berhasil dimuat: {len(labels)} item")
        else:
            print(f"⚠️ File label {key_name} tidak ditemukan atau path kosong: {path}")

    # --- Simpan merge label ---
    all_label_path = cfg.get('all_label_path')
    os.makedirs(os.path.dirname(all_label_path), exist_ok=True)
    with open(all_label_path, 'w', encoding='utf-8') as f:
        json.dump(all_labels, f, indent=2, ensure_ascii=False)

    print(f"📝 Merge label selesai, total {len(all_labels)} label disimpan di: {all_label_path}")
    return all_labels

# --- Jalankan merge ---
all_labels = merge_labels(CFG)


⚠️ File label PlantVillage tidak ditemukan atau path kosong: /content/drive/MyDrive/AgrifyAI/models/plantvillage_labels.json
⚠️ File label Paddy tidak ditemukan atau path kosong: /content/drive/MyDrive/AgrifyAI/models/paddy_labels.json
📝 Merge label selesai, total 0 label disimpan di: /content/drive/MyDrive/AgrifyAI/models/labels.json


In [8]:
import json
import os

def clean_and_translate_labels(cfg):
    """
    Load merged labels (PlantVillage + Paddy), translate ke Bahasa Indonesia,
    dan simpan ke translated_label_path (labels_id.json)
    """
    original_label_path = cfg["all_label_path"]
    translated_label_path = cfg["translated_label_path"]

    # === KAMUS TERJEMAHAN PLANTVILLAGE + PADDY ===
    full_phrase_map = {
        # PlantVillage
        'apple___apple_scab': 'Apel - Kudis Apel',
        'apple___black_rot': 'Apel - Busuk Hitam',
        'apple___cedar_apple_rust': 'Apel - Karat Apel Cedar',
        'apple___healthy': 'Apel - Sehat',
        'blueberry___healthy': 'Bluberi - Sehat',
        'cherry___powdery_mildew': 'Ceri - Embun Tepung',
        'cherry___healthy': 'Ceri - Sehat',
        'corn___cercospora_leaf_spot_gray_leaf_spot': 'Jagung - Bercak Daun Cercospora',
        'corn___common_rust': 'Jagung - Karat Umum',
        'corn___northern_leaf_blight': 'Jagung - Hawar Daun Utara',
        'corn___healthy': 'Jagung - Sehat',
        'grape___black_rot': 'Anggur - Busuk Hitam',
        'grape___esca_(black_measles)': 'Anggur - Esca (Cacar Hitam)',
        'grape___leaf_blight_(isariopsis_leaf_spot)': 'Anggur - Hawar Daun Isariopsis',
        'grape___healthy': 'Anggur - Sehat',
        'orange___haunglongbing_(citrus_greening)': 'Jeruk - Penyakit CVPD',
        'peach___bacterial_spot': 'Persik - Bercak Bakteri',
        'peach___healthy': 'Persik - Sehat',
        'pepper,_bell___bacterial_spot': 'Paprika - Bercak Bakteri',
        'pepper,_bell___healthy': 'Paprika - Sehat',
        'potato___early_blight': 'Kentang - Hawar Daun Dini',
        'potato___late_blight': 'Kentang - Hawar Daun Akhir',
        'potato___healthy': 'Kentang - Sehat',
        'raspberry___healthy': 'Raspberry - Sehat',
        'soybean___healthy': 'Kedelai - Sehat',
        'squash___powdery_mildew': 'Labu - Embun Tepung',
        'strawberry___leaf_scorch': 'Stroberi - Daun Gosong',
        'strawberry___healthy': 'Stroberi - Sehat',
        'tomato___bacterial_spot': 'Tomat - Bercak Bakteri',
        'tomato___early_blight': 'Tomat - Hawar Daun Dini',
        'tomato___late_blight': 'Tomat - Hawar Daun Akhir',
        'tomato___leaf_mold': 'Tomat - Jamur Daun',
        'tomato___septoria_leaf_spot': 'Tomat - Bercak Daun Septoria',
        'tomato___spider_mites_two-spotted_spider_mite': 'Tomat - Tungau Laba-laba',
        'tomato___target_spot': 'Tomat - Bercak Target',
        'tomato___tomato_yellow_leaf_curl_virus': 'Tomat - Virus Keriting Daun Kuning',
        'tomato___tomato_mosaic_virus': 'Tomat - Virus Mosaik',
        'tomato___healthy': 'Tomat - Sehat',
        # Paddy
        'normal': 'Padi - Sehat',
        'blast': 'Padi - Hawar Daun Blast',
        'hispa': 'Padi - Wereng Batang',
        'dead_heart': 'Padi - Dead Heart (Hama/Tepi Daun Mati)',
        'tungro': 'Padi - Virus Tungro',
        'brown_spot': 'Padi - Bercak Coklat',
        'downy_mildew': 'Padi - Embun Tepung Daun',
        'bacterial_leaf_blight': 'Padi - Hawar Daun Bakteri',
        'bacterial_leaf_streak': 'Padi - Garis Bakteri Daun',
        'bacterial_panicle_blight': 'Padi - Hawar Daun Penicle Bakteri'
    }

    # Normalisasi key supaya case-insensitive
    full_phrase_map = {k.lower(): v for k, v in full_phrase_map.items()}

    try:
        with open(original_label_path, 'r', encoding='utf-8') as f:
            all_labels = json.load(f)

        translated_labels = {}
        for k, v in all_labels.items():
            key = v.lower().replace(' ', '_')
            translated_labels[k] = full_phrase_map.get(key, v.replace('_', ' ').capitalize())

        # Simpan hasil translate
        with open(translated_label_path, 'w', encoding='utf-8') as f:
            json.dump(translated_labels, f, indent=2, ensure_ascii=False)

        print(f"✅ Label berhasil diterjemahkan dan disimpan di: {translated_label_path}")
        CFG["translated_label_path"] = translated_label_path

    except FileNotFoundError:
        print(f"⚠️ File '{original_label_path}' tidak ditemukan.")
    except Exception as e:
        print(f"❌ Terjadi kesalahan: {e}")

# --- Jalankan ---
clean_and_translate_labels(CFG)


✅ Label berhasil diterjemahkan dan disimpan di: /content/drive/MyDrive/AgrifyAI/models/labels_id.json


## Step 5: Defining Models

In [9]:
# ==============================================================================
# SEL 3 (REFAC SESUAI CFG): DEFINE MODEL, TRANSFER LEARNING, FINE-TUNING, CALLBACKS
# ==============================================================================

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os

# -------------------------------
# Build Transfer Learning Model
# -------------------------------
def build_transfer_model(num_classes, weights="imagenet"):
    """
    Build MobileNetV3Large dengan head baru.
    """
    input_shape = CFG["img_size"] + (3,)

    # Base model
    base_model = tf.keras.applications.MobileNetV3Large(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet"
    )

    # Freeze backbone awal
    base_model.trainable = False

    # Build head
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(num_classes, activation='softmax', dtype='float32', name='predictions')(x)

    model = models.Model(inputs=base_model.input, outputs=output)
    model.base_model = base_model

    print(f"✅ Transfer model dibangun (backbone=MobileNetV3, kelas={num_classes})")
    return model


# -------------------------------
# Fine-tuning helper
# -------------------------------
def fine_tune_model(model, fine_tune_at=CFG["fine_tune_at"]):
    base_model = getattr(model, "base_model", None)
    if base_model is None:
        print("⚠️ Base model tidak ditemukan, fine-tuning dibatalkan.")
        return model

    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    print(f"🔓 Unfreeze mulai dari layer ke-{fine_tune_at}")
    return model

# -------------------------------
# Callbacks
# -------------------------------
def get_callbacks():
    return [
        EarlyStopping(monitor='val_loss', patience=CFG['patience'], restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=CFG['min_lr']),
        ModelCheckpoint(CFG['model_path'], monitor='val_loss', save_best_only=True, verbose=1)
    ]

# -------------------------------
# Contoh penggunaan (HARAP PERHATIKAN SEBELAH SINI JIKA MAU TRAIN, SESUAIKAN WEIGHTS!)
# -------------------------------
num_classes = 48  # ganti sesuai dataset
model = build_transfer_model(num_classes, weights="imagenet")
model = fine_tune_model(model, fine_tune_at=CFG["fine_tune_at"])
model.summary()


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
✅ Transfer model dibangun (backbone=MobileNetV3, kelas=48)
🔓 Unfreeze mulai dari layer ke-100


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation[0][0]  │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu[0][0]       │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation[0][0], │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_1[0][0]     │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,562,288 (13.59 MB)

 Trainable params: 3,376,912 (12.88 MB)

 Non-trainable params: 185,376 (724.12 KB)

## Step 6: Training Phase 1

In [ ]:
def train_phase_1(train_dataset, val_dataset, num_classes):
    print("--- 🚀 PHASE 1: TRAIN HEAD ONLY ---")

    weights_path = CFG["weights_path"]  # simpan sini
    model_path = CFG["model_path"]

    # Kalau weights ada, load dulu
    if os.path.exists(weights_path):
        print(f"🔄 Load weights dari: {weights_path}")
        model = build_transfer_model(num_classes, weights="imagenet")  # build dulu
        model.load_weights(weights_path)
    else:
        print("⚠️ Weights tidak ditemukan, build model baru dari ImageNet")
        model = build_transfer_model(num_classes, weights="imagenet")

    # Freeze base_model
    if hasattr(model, "base_model"):
        model.base_model.trainable = False
    else:
        print("⚠️ Base model tidak ditemukan, skip freeze")

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=CFG["lr"]),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train head
    model.fit(
        train_dataset,
        validation_data=val_dataset,
        # epochs=CFG["epochs"],
        epochs=5,
        callbacks=get_callbacks()
    )

    # Simpan hanya weights (.h5)
    model.save_weights(weights_path)
    print(f"✅ Phase 1 selesai. Weights disimpan di: {weights_path}")

    return model

# Contoh pemanggilan
train_phase_1(train_dataset, val_dataset, num_classes)


--- 🚀 PHASE 1: TRAIN HEAD ONLY ---
⚠️ Weights tidak ditemukan, build model baru dari ImageNet
✅ Transfer model dibangun (backbone=MobileNetV3, kelas=48)
Epoch 1/5
    335/Unknown 768s 2s/step - accuracy: 0.3767 - loss: 2.6669

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 1: val_loss improved from inf to 1.07199, saving model to /content/drive/MyDrive/AgrifyAI/models/plant_disease_model_L4.keras
336/336 ━━━━━━━━━━━━━━━━━━━━ 1023s 3s/step - accuracy: 0.3773 - loss: 2.6640 - val_accuracy: 0.7031 - val_loss: 1.0720 - learning_rate: 1.0000e-04
Epoch 2/5
228/336 ━━━━━━━━━━━━━━━━━━━━ 3:45 2s/step - accuracy: 0.7630 - loss: 0.9021

## Step 7: Training Phase 2

In [ ]:
def train_phase_2(train_dataset, val_dataset, num_classes):
    print("--- 🚀 PHASE 2: FINE-TUNING ---")

    weights_path = CFG["weights_path"]
    model_path = CFG["model_path"]

    # Build model baru dari scratch
    model = build_transfer_model(num_classes, weights="imagenet")

    # Load weights .h5 kalau ada
    if os.path.exists(weights_path):
        print(f"🔄 Load weights dari: {weights_path}")
        model.load_weights(weights_path)
    else:
        print(f"⚠️ Weights .h5 tidak ditemukan di {weights_path}, lanjut tanpa load")

    # Unfreeze sebagian backbone
    if hasattr(model, "base_model"):
        model.base_model.trainable = True
        fine_tune_at = CFG.get("fine_tune_at", 100)
        for layer in model.base_model.layers[:fine_tune_at]:
            layer.trainable = False
        print(f"🔓 Unfreeze mulai dari layer ke-{fine_tune_at}")
    else:
        print("⚠️ Base model tidak ditemukan, skip unfreeze")

    # Compile dengan lr kecil
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=CFG.get("fine_tune_lr", 1e-5)),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train
    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=CFG.get("fine_tune_epochs", 10),
        callbacks=get_callbacks()
    )

    # Save full model .keras
    model.save(model_path)
    print(f"✅ Phase 2 selesai. Model disimpan di: {model_path}")

    return model

# Pemanggilan
train_phase_2(train_dataset, val_dataset, num_classes)


In [ ]:
!pip install --force-reinstall "openvino==2024.4.0" "openvino-dev==2023.3.0" --no-deps

In [ ]:
!pip install -q tf2onnx

## Step 8: Convert to OpenVINO

In [ ]:
# ==============================================================================
# PIPELINE: WRAP + SAVE + CONVERT TO OPENVINO (Colab-ready)
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.layers as keras_layers
from pathlib import Path
import json, os

# OpenVINO converter
from openvino.tools import mo

# ---------------------------------------------------------------------
# Helper: detect custom objects (layers & Lambda functions)
# ---------------------------------------------------------------------
def detect_custom_objects(model):
    custom_objects = {}
    keras_layer_classes = tuple(
        v for k,v in keras_layers.__dict__.items() if isinstance(v, type)
    )

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Lambda):
            try:
                func = layer.function
                custom_objects[layer.name] = func
            except Exception:
                custom_objects[layer.name] = layer.__class__
        else:
            if not isinstance(layer, keras_layer_classes):
                custom_objects[layer.name] = layer.__class__

    return custom_objects


# ---------------------------------------------------------------------
# Safe wrapper: do NOT call old_model(input) — just re-use graph
# ---------------------------------------------------------------------
def wrap_model_for_export_safe(model_path, custom_objects=None):
    custom_objects = custom_objects or {}

    try:
        old_model = load_model(model_path, compile=False)
    except Exception:
        old_model = load_model(model_path, compile=False, custom_objects=custom_objects)

    # Auto-detect custom objects
    auto = detect_custom_objects(old_model)
    if auto:
        print("⚠️ Auto-detected custom objects / Lambda layers:", list(auto.keys()))
        used_custom_objects = {**custom_objects, **auto}
        old_model = load_model(model_path, compile=False, custom_objects=used_custom_objects)
    else:
        used_custom_objects = custom_objects

    # Wrap model safely
    wrapped_model = Model(inputs=old_model.inputs, outputs=old_model.outputs, name="wrapped_model_safe")

    # Handle multi-input/output print
    def shape_or_list(x):
        if isinstance(x, (list, tuple)):
            return [t.shape for t in x]
        return x.shape

    print(f"✅ Wrapped model ready.")
    print(f"   Input shape(s): {shape_or_list(wrapped_model.inputs)}")
    print(f"   Output shape(s): {shape_or_list(wrapped_model.outputs)}")

    return wrapped_model, used_custom_objects


# ---------------------------------------------------------------------
# PIPELINE FUNCTION
# ---------------------------------------------------------------------
def export_and_convert_to_openvino(model_path, out_dir):
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # 1. Wrap safe
    wrapped_model, used_custom = wrap_model_for_export_safe(model_path)

    # 2. Save as TensorFlow SavedModel (.pb)
    savedmodel_dir = os.path.join(out_dir, "wrapped_savedmodel")
    wrapped_model.export(savedmodel_dir)   # <-- pakai export(), bukan save()
    print("✅ Wrapped model exported (SavedModel) at:", savedmodel_dir)

    # 3. Save as H5/keras (optional backup)
    wrapped_h5 = os.path.join(out_dir, Path(model_path).stem + ".wrapped.h5")
    wrapped_model.save(wrapped_h5, include_optimizer=False)  # save_format ga perlu
    print("✅ Saved as HDF5 (.h5) at:", wrapped_h5)

    # 4. Dump custom objects for debug
    custom_json_path = os.path.join(out_dir, "detected_custom_objects.json")
    with open(custom_json_path, "w") as f:
        json.dump({k: str(v) for k, v in used_custom.items()}, f, indent=2)
    print("✅ Custom objects dumped at:", custom_json_path)

    # 5. Convert SavedModel to OpenVINO IR
    print("⚡ Converting to OpenVINO IR ...")
    ov_model = mo.convert_model(savedmodel_dir)

    # Save IR (.xml + .bin)
    openvino_path = os.path.join(out_dir, "openvino_model.xml")
    from openvino.runtime import serialize
    serialize(ov_model, openvino_path)
    print("✅ OpenVINO IR model saved at:", openvino_path)

    return openvino_path



# ---------------------------------------------------------------------
# Example usage (Colab)
# ---------------------------------------------------------------------

openvino_model_path = export_and_convert_to_openvino(CFG["model_path"], CFG["gdrive_model_dir"])
print("🎉 Done! Your OpenVINO model is at:", openvino_model_path)


## Step 9: Testing

In [ ]:
from openvino.runtime import Core
import numpy as np
import json

def run_openvino_inference_sample(val_dataset, xml_model_path, labels_path, buffer_size=1000):
    """Memuat model OpenVINO IR dan menjalankan inferensi pada 1 sampel acak."""
    print("\n--- ⚡ Menjalankan Inferensi dengan OpenVINO Runtime ---")
    core = Core()
    model = core.read_model(model=xml_model_path)
    compiled_model = core.compile_model(model=model, device_name="CPU")

    input_layer = compiled_model.input(0)
    output_layer = compiled_model.outputs[0]

    # Load labels sekali di awal
    with open(labels_path, 'r') as f:
        labels_dict = json.load(f)

    # Ambil 1 sampel acak dari val_dataset
    for image, label in val_dataset.shuffle(buffer_size=buffer_size).take(1):
        image_tensor = np.expand_dims(image[0].numpy(), axis=0).astype(np.float32)
        result = compiled_model([image_tensor])[output_layer]
        pred_id = int(np.argmax(result))
        true_id = int(label[0].numpy())

        # Ambil key string sesuai index
        true_key = list(labels_dict.keys())[true_id]
        pred_key = list(labels_dict.keys())[pred_id]

        print("\n--- Hasil Prediksi ---")
        print(f"Label Asli        : {true_key} ({labels_dict[true_key]})")
        print(f"Prediksi OpenVINO : {pred_key} ({labels_dict[pred_key]})")
        print("🎉 BENAR!" if pred_id == true_id else "🤔 SALAH")


# ----------------------------- # Eksekusi pipeline # -----------------------------
xml_path = CFG["openvino_path_xml"]
run_openvino_inference_sample(val_dataset, xml_path, CFG["translated_label_path"])